## Shear Dispersion Initial Value Problem Solution with Localized Initial Condition

The differential equation is
\begin{equation}
\epsilon\frac{\partial\theta}{\partial t} + \cos{(y)}\frac{\partial\theta}{\partial x} = \frac{1}{Pe}\frac{\partial^2\theta}{\partial x^2} + \epsilon\frac{\partial^2\theta}{\partial y^2} ,
\end{equation}

with an initial condition given by a Gaussian centered at $x=0$.
The problem is solved by synthesizing the initial condition as a Fourier cosine series and using the exact solution for a cosine initial condition (see `Adv_Diff_IVP_01.ipynb`).



In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')

In [ ]:
from mathieu_functions import A_coefficients
from mathieu_functions import mathieu_functions as mfs

In [ ]:
# =================================
# Important parameters to define
# =================================
M = 1 # Channel width (y-direction)
alpha = 25  # length of channel periodic in x
Nk = 50  # Length of k-array (Truncated x-Fourier series of the initial condition).
Nx = 10 * Nk  # Grid resolution in x.
sigma = 1  # Width of gaussian

Pe = 200

x = np.linspace(-alpha * (np.pi * M), alpha * (np.pi * M), Nx)
y = np.linspace(0, (np.pi * M), Nx // 5)  # \tilde{y} in paper.
X, Y = np.meshgrid(x, y)

K = np.arange(0, Nk / alpha, 1 / alpha)  # wavenumber array.
Q = (1j) * 2 * K * Pe  # Canonical Mathieu parameter
qf = Q[-1].imag  # Largest value of Mathieu's parameter. 

# =============================
N = 50  # matrix size --- determines the number of eigenfunctions and eigenvalues in the truncated system
# =============================

t = np.linspace(0, .25, 100)

In [ ]:
K

In [ ]:
if qf > 1000:
    print('Value of parameter q is:', (qf * (1j)))
    raise Warning('Change either epsilon or k, to reduce the size of q. The current code only works for values q>1000i')
print('Value of parameter q is:', (qf * (1j)))

In [ ]:
A_vals = A_coefficients(Q, N, 'even', 'one')
vals = mfs.ce_even(Q, y, N, As=A_vals)

## Mathieu functions

Creates a list with the right size

In [ ]:
CE = []  # Initialize list containing Mathieu functions
for k in range(N // 2):
    ce = np.repeat(vals['ce'+str(2 * k)][:, :, np.newaxis], Nx, axis=2)
    CE.append(ce)

COS = [np.exp(K[i] * X *(1j)) for i in range(Nk)]

In [ ]:
del ce

## Fourier Coefficients for the $x$-Fourier approximation of the initial condition

In [ ]:
fac = sigma/alpha
cn = []
for n in range(Nk):
    cn.append(fac * np.exp(-((sigma) * K[n])**2))

## Define a function to construct the solution 

In [ ]:
def evolve_ds(As, CE, K, cn, sigma, X, Y, t):
    """Constructs the solution to the IVP"""
    ## Initialize the array
    coords = {"time": t, 
              "y": 2 * Y[:, 0], 
              "x": X[0, :]}
    Temp = xr.DataArray(np.nan, coords=coords, dims=["time", 'y', 'x'])
    ds = xr.Dataset({'Theta': Temp})
    N = len(K)
    for i in range(len(t)):
        print(i)
        coeff=[]
        for k in range(N):
            CE2n = [2 * As['A'+str(2*r)][k, 0] * CE[r][k, :, :] * np.exp(-(0.25*As['a'+str(2*r)][k] + K[k]**2)*t[i]) for r in range(len(CE))]
            CE2n = sum(CE2n) # r-sum
            coeff.append(cn[k] * CE2n * COS[k])
        T0 = np.sum(coeff, axis=0).real # k-sum
        ds['Theta'].data[i, :, :] = T0
    return ds

## Construct solution

Can take some time. Counter must reach 100


In [ ]:
ds = evolve_ds(A_vals, CE, K, cn, sigma, X, Y, t)

In [ ]:
del CE  # frees up memory

## Plot animation

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='nipy_spectral') plot[colorbar=True]
%%opts Image [width=600, height=450]
hv_ds = hv.Dataset(ds.Theta)
hv_ds.to(hv.Image, ['x', 'y'])

## Now calculate the cross-channel mean evolution

In [ ]:
Time, Xt = np.meshgrid(x, t)

In [ ]:
Tm = np.trapz(ds.Theta, axis=1) * y[1] / np.pi

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8), facecolor='w')
cf=plt.contourf(Time, Xt, Tm,  levels=np.linspace(0, 1, 100), cmap='nipy_spectral')
plt.xticks(size=15)
plt.yticks([0, .1, .2], size=15)
plt.ylim(0, 0.2)
plt.xlabel('x', fontsize=25)
plt.xlim(x[0], x[-1])
plt.ylabel(r'$t$', fontsize=35, rotation=0, labelpad = 35)
cbaxes = fig.add_axes([0.675, 0.935, 0.225, 0.03])
clb1 = plt.colorbar(cf,cax=cbaxes,ticks=[0, 0.5, 1],orientation='horizontal')
clb1.ax.tick_params(labelsize=15)
plt.show()